In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from google.colab import auth
import os
auth.authenticate_user()
from google.cloud import bigquery
!gcloud config set project hmrecommendation


Updated property [core/project].


In [ ]:
project_id = 'hmrecommendation'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

In [ ]:
client = bigquery.Client(project=project_id)

In [ ]:
create_joined_query = """
select * from `hmrecommendation.hm_dataset.transactions` a 
left join `hmrecommendation.hm_dataset.customers` b using (customer_id)
left join `hmrecommendation.hm_dataset.articles` c using (article_id)
"""
job_config = bigquery.QueryJobConfig(destination='hmrecommendation.hm_dataset.trans_cust_article_join')
qeury_job = client.query(create_joined_query, job_config= job_config)
qeury_job.result()


In [ ]:
table = client.get_table('hmrecommendation.hm_dataset.trans_cust_article_join_copy')
original_schema = table.schema
new_schema = original_schema[:]  # Creates a copy of the schema.
new_schema.append(bigquery.SchemaField("uuid", "STRING"))
table.schema = new_schema
table = client.update_table(table, ["schema"]) 
if len(table.schema) == len(original_schema) + 1 == len(new_schema):
    print("A new column has been added.")
else:
    print("The column has not been added.")

In [ ]:
insert_uuid = """insert hmrecommendation.hm_dataset.trans_cust_article_join_copy` (uuid) values (SELECT GENERATE_UUID() AS uuid) """


In [ ]:
query_job = client.query(insert_uuid)

In [ ]:
create_validation = '''
with a1 as (
select *, ROW_NUMBER() over (partition by customer_id order by t_dat) rn
from (select customer_id, count(customer_id) as count from `hmrecommendation.hm_dataset.trans_cust_article_join`  group by customer_id having count(customer_id) > 1) a 
left join (select * from `hmrecommendation.hm_dataset.trans_cust_article_join`) b using (customer_id)
)
SELECT * FROM a1 WHERE rn = 1;
'''
job_config = bigquery.QueryJobConfig(destination='hmrecommendation.hm_dataset.validation_dataset')
query_job = client.query(create_validation, job_config=job_config)  # Make an API request.
query_job.result() 

In [ ]:
# drop_validation = '''
# delete `hmrecommendation.hm_dataset.trans_cust_article_join` a 
# where a.customer_id in (select customer_id from `hmrecommendation.hm_dataset.validation_dataset`) 
# and a.t_dat in (select t_dat from `hmrecommendation.hm_dataset.validation_dataset`)
# '''

In [ ]:
# query_job = client.query(drop_validation)
# query_job.result() 

In [ ]:
query_data_sample = """
SELECT * FROM `hmrecommendation.hm_dataset.trans_cust_article_join` LIMIT 10000
"""

In [ ]:
sample_joined = client.query(query = query_data_sample).to_dataframe()

In [ ]:
sample_joined.head()

,article_id,customer_id,t_dat,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,917297003,b0e78def704db4d6c303c0e66cb852e93b8f66818b8092...,2020-09-11,0.059305,2,NaN,NaN,ACTIVE,NONE,26.0,9815e87fff4be4652534a8410fb284945779bba631f6a2...,917297,BLANKS JERRY OVERSIZED HOOD,308,Hoodie,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,5888,Blanks,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic,Blank Staples is a streetwear collection of mi...
1,917296004,85156e749dd36c74ee49c0accd7574a1388adfad87480c...,2020-09-18,0.050831,2,NaN,NaN,ACTIVE,NONE,56.0,813dd6acc5814a0432626bcf38e0771983a9814ac142dd...,917296,BLANKS JIM OVERSIZED SWEATER,252,Sweater,Garment Upper body,1010016,Solid,14,Dark Beige,4,Dark,11,Beige,5888,Blanks,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic,Blank Staples is a streetwear collection of mi...
2,709749002,46948093bae7d677999b48a5c18c79021160191f1281a7...,2019-07-15,0.042695,2,NaN,NaN,ACTIVE,NONE,26.0,b92313f3c4bcb1cb75aeb743f7d426ccec98aa312b8348...,709749,2pk TOM,59,Swimwear bottom,Swimwear,1010012,Mixed solid/pattern,91,Light Green,3,Light,19,Green,9984,Swimwear,F,Menswear,3,Menswear,26,Men Underwear,1018,Swimwear,Swim shorts in woven fabric with an elasticate...
3,709749003,8a80d539ab918824159dd728c563af916bf685561d9800...,2020-08-05,0.047441,2,NaN,NaN,ACTIVE,NONE,58.0,e07036f48f4d10072ddf75b125bd11e6ddb6b62c500c78...,709749,2pk TOM,59,Swimwear bottom,Swimwear,1010012,Mixed solid/pattern,73,Dark Blue,4,Dark,2,Blue,9984,Swimwear,F,Menswear,3,Menswear,26,Men Underwear,1018,Swimwear,Swim shorts in woven fabric with an elasticate...
4,854328001,d47058cfdc850e954394829ab814f54914c4664f14f57a...,2020-06-17,0.022017,2,1.0,1.0,ACTIVE,Regularly,20.0,9eb19de7be9b1c90b598fadfd6dd5240d05a7343ccf387...,854328,2pk BRAD,59,Swimwear bottom,Swimwear,1010001,All over pattern,92,Green,5,Bright,19,Green,9984,Swimwear,F,Menswear,3,Menswear,26,Men Underwear,1018,Swimwear,Swim shorts in recycled polyester with an elas...


In [ ]:
sample_joined.shape

(10000, 35)

In [ ]:
# customer_sample_query = """
# SELECT * FROM `hmrecommendation.hm_dataset.customers` LIMIT 10
# """
# customer_sample = client.query(query = customer_sample_query).to_dataframe()

In [ ]:
# def get_test(all_data):
#   subset = all_data.loc[:, ['customer_id', 't_dat']].groupby('customer_id').size().reset_index()
#   re_purchase = all_data.loc[all_data['customer_id'].isin(subset.loc[subset[0] > 1, 'customer_id'])]
#   re_purchase = re_purchase.sort_values(by = 't_dat').drop_duplicates('customer_id',keep='last')
#   return re_purchase

In [ ]:
# re_purchased = get_test(sample_joined)

In [ ]:
# re_purchased

In [ ]:
# item_sample_query = """
# SELECT * FROM `hmrecommendation.hm_dataset.articles` LIMIT 10
# """
# item_sample = client.query(query = item_sample_query).to_dataframe()

cleaning

In [ ]:
def cleaning(data, dict):
  for k in dict:
    v = dict.get(k)
    data.loc[:,k].fillna(v, inplace = True)
  return data

In [ ]:
sample_joined.columns

Index(['article_id', 'customer_id', 't_dat', 'price', 'sales_channel_id', 'FN',
       'Active', 'club_member_status', 'fashion_news_frequency', 'age',
       'postal_code', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [ ]:
sample_joined = cleaning(sample_joined, {'FN': 0, 'Active':0, 'club_member_status': 'Null', 'fashion_news_frequency': 'None', 'detail_desc': 'No Desc'})
sample_joined.loc[:,'fashion_news_frequency'].replace('NONE', 'None', inplace = True)
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
sample_joined.age = imp.fit_transform(np.array(sample_joined.age).reshape(-1,1))

In [ ]:
##encoded_customer_df = sample_joined.copy().loc[:, ['FN', 'Active','club_member_status','fashion_news_frequency', 'age']]

In [ ]:
no_name_col = ['article_id', 'customer_id', 't_dat', 'price', 'sales_channel_id', 'FN',
       'Active', 'club_member_status', 'fashion_news_frequency', 'age',
       'postal_code', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no', 'colour_group_code',
       'perceived_colour_value_id',
       'perceived_colour_master_id', 
       'department_no', 'index_code',
       'index_group_no', 'section_no',
       'garment_group_no' ]

In [ ]:
X = sample_joined.loc[:,no_name_col]
X = X.drop(['article_id', 'customer_id', 't_dat'], axis = 1)
y = pd.DataFrame(sample_joined.article_id)

In [ ]:
X.shape

(10000, 22)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state = 42 )

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

def encodeCategory(data):
  encoder = LabelEncoder()
  encoded_sample_join = data.copy()
  for i in range(len(data.columns)):
    if encoded_sample_join.iloc[:,i].dtypes == object:
      encoded_sample_join.iloc[:,i] = encoder.fit_transform(encoded_sample_join.iloc[:,i])
  return encoded_sample_join

In [ ]:
X_train = encodeCategory(X_train)
X_test = encodeCategory(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

def scaleData(data):
  return StandardScaler().fit_transform(data)

In [ ]:
X_train = scaleData(X_train)
X_test = scaleData(X_test)

In [ ]:
##encoded_sample_join_scaled = pd.DataFrame(encoded_sample_join,columns = data.columns)

In [ ]:
from sklearn.decomposition import PCA
def createPCA(data):
  pca = PCA(0.95)
  PCA_result = pca.fit_transform(data)
  PCA_result_df = pd.DataFrame(PCA_result)
  return PCA_result_df

In [ ]:
X_train_PCA = createPCA(X_train)
X_test_PCA = createPCA(X_test)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
##sns.pairplot(PCA_result_df)

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
##PCA_result_df.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

wcss = []
for i in range(1, 31):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X_train_PCA)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 31), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show

KeyboardInterrupt: ignored

In [ ]:
y_pred = kmeans.fit(X_train_PCA).predict(X_test_PCA)

In [ ]:
y_test['pred_cluster'] = y_pred

In [ ]:
y_test.groupby('pred_cluster').size().sort_values()

pred_cluster
2      2
6     30
8     49
4     72
7    312
3    484
1    516
0    556
9    569
5    710
dtype: int64

In [ ]:
y_test

,article_id,pred_cluster
6252,759637009,0
4684,537896003,3
1731,689389001,0
4742,200182001,7
4521,575347003,3
...,...,...
1744,689389002,9
9754,864380002,5
6094,775977002,4
8781,817361002,5


In [ ]:
## join cluster back to the orginal dataframe based on articles_id
## join valuable customer to the original dataframe
## use cluster and valuable customer tag to predict future purchase